### requirements for the following codings


In [ ]:
### packages required
!pip install fair-esm

In [ ]:
import numpy as np
import pandas as pd
import esm
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D
from keras.models import Sequential,Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras
from keras import backend as K
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU') # set the deep learning with GPU
else:
    print("No GPU found")

### peptide embeddings with esm2_t30_150M_UR50D pretrained models
30 layers, 150M parameters, dataset: UR50/D 2021_04, embedding dimension: 640

original ESM github site is https://github.com/facebookresearch/esm

In [ ]:
def esm_embeddings(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting, (i personally suggest to do the embedding one by one if you are running locally or with T4 GPU at google colab)
  #         you computer might automatically kill the job.
  # load the python packages
  import torch
  import esm
  import collections
  import pandas as pd
  import gc
  # choose cuda for acceleration if available
  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2.eval()
  esm2 = esm2.to(device)
  batch_converter = esm2_alphabet.get_batch_converter() # this function is to tokenize your peptide sequences from amino acid sequence into numbers

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the tokenized results of the whole data set
  batch_tokens = batch_tokens.to(device)
  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      # if you want to change the esm model, for example: esm2_t12_35M_UR50D, which has 12 layers, they you will need to change the 6 to 12 at the following two line
      results = esm2(batch_tokens, repr_layers=[30], return_contacts=False)
  token_representations = results["representations"][30].cpu() # the representation is generated for each amino acid residue; for example, if you have 3 residues, then the ourput shape is 3 * 320, 3 is the number of residues and the 320 is the vector for a single rediue

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so we will discard it and start from token 1
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor format can be transformed as numpy format sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations # delete those variables to save GPU memory
  gc.collect() # release the GPU memory
  return embeddings_results


### data loading and embeddings
assume you have already split your dataset as a train and a test dataset.

In [ ]:
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [ ]:
# training dataset loading
dataset = pd.read_excel('bitter_train.xlsx',header=0, index_col = None)
# generate embedding for seqeunces
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# test dataset loading
dataset = pd.read_excel('bitter_test.xlsx',header=0, index_col = None)
# generate embedding for seqeunces
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq] )
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')

#### dataset division
ready for training and evaluation

In [ ]:
# training dataset loading
dataset = pd.read_excel('bitter_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence']
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

# test dataset loading
dataset = pd.read_excel('bitter_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence']
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

# loading the peptide embddings
X_train = pd.read_csv('train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv',header=0, index_col = 0,delimiter=',')
X_test = pd.read_csv('test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv',header=0, index_col = 0,delimiter=',')
# generate the validation dataset for early stopping
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=123, shuffle= True, stratify =y_train )

# transform the dataset into numpy format
X_train = np.array(X_train)
X_valid = np.array(X_valid)
X_test = np.array(X_test)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_valid = scaler.transform(X_valid) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

In [ ]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

### dataset loading and embeddings
(if you only have one dataset and need to split them)

In [ ]:
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [ ]:
# whole dataset loading and dataset splitting
dataset = pd.read_excel('bitter_single_dataset_example.xlsx',header=0, index_col = None)

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv')


#### dataset division
ready for training and evaluation

In [ ]:
# loading the y dataset for model development
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data = pd.read_csv('whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv',header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123, shuffle= True, stratify =y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=234, shuffle= True, stratify =y_train)


# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

### Model architecture

In [ ]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  inputShape=(640,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(32,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  adam = Adam(learning_rate=0.001)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model.h5',  monitor='val_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### model training and evaluation at test dataset

In [ ]:
# train the model & best model checkpoint will be save as 'best_model.h5'
model, model_history = ESM_CNN(X_train, y_train, X_valid, y_valid)

In [ ]:
# load the saved best model for performance evaluation at test dataset
saved_model = load_model('best_model.h5')
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
recall_collecton = []
precision_collecton = []
f1_collecton = []
MCC_collecton = []
AUC_collecton = []
# confusion matrix
predicted_class= []
predicted_protability = saved_model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0] # the class with higher protability was treated as the prediction class
  predicted_class.append(index)

predicted_class = np.array(predicted_class) # transformed into a numpy for performance evaluation
y_true = y_test
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
recall_collecton.append(TP/(TP+FN))
precision_collecton.append(TP/(TP+FP))
f1_collecton.append(2*TP/(TP+FP)*TP/(TP+FN)/ (TP/(TP+FP)+TP/(TP+FN)))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
# print the performance in the test dataset
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(recall_collecton[0])
print(precision_collecton[0])
print(f1_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

### model usage at new dataset

In [ ]:
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
# new dataset loading
dataset = pd.read_excel('bitter_new_data.xlsx',header=0, index_col = None)
# generate embedding for seqeunces
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('new_data_esm2_t6_8M_UR50D_unified_320_dimension.csv')


In [ ]:
# loading scaler from new dataset
X_train = pd.read_csv('train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv',header=0, index_col = 0,delimiter=',')
# transform the dataset into numpy format
X_train = np.array(X_train)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
# loading the peptide embddings
X_new_embeddings = pd.read_csv('new_data_esm2_t6_8M_UR50D_unified_320_dimension.csv',header=0, index_col = 0,delimiter=',')
# transform the dataset into numpy format
X_new = np.array(X_new_embeddings)
X_new = scaler.transform(X_new) # normalize X to 0-1 range

In [ ]:
# load the saved best model for performance evaluation at test dataset
saved_model = load_model('best_model.h5')
# confusion matrix
predicted_class= []
predicted_protability = saved_model.predict(X_new,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
for i in range(dataset.shape[0]):
  dataset.iloc[i,1] = predicted_class[i]
dataset.to_excel('new_data_prediction_result.xlsx')